In [4]:
import argparse
import os, sys
import os.path as osp
import torchvision
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import network, loss
from torch.utils.data import DataLoader
from data_list import ImageList, ImageList_idx
import random, pdb, math, copy
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
from train_tar import *

In [5]:
parser = argparse.ArgumentParser(description='Neighbors')
parser.add_argument('--gpu_id',
                    type=str,
                    nargs='?',
                    default='1',
                    help="device id to run")
parser.add_argument('--s', type=int, default=0, help="source")
parser.add_argument('--t', type=int, default=1, help="target")
parser.add_argument('--max_epoch',
                    type=int,
                    default=15,
                    help="max iterations")
parser.add_argument('--interval', type=int, default=15)
parser.add_argument('--batch_size',
                    type=int,
                    default=64,
                    help="batch_size")
parser.add_argument('--worker',
                    type=int,
                    default=4,
                    help="number of workers")
parser.add_argument(
    '--dset',
    type=str,
    default='visda-2017')
parser.add_argument('--lr', type=float, default=1e-3, help="learning rate")
parser.add_argument('--net',
                    type=str,
                    default='resnet101')
parser.add_argument('--seed', type=int, default=2021, help="random seed")

parser.add_argument('--bottleneck', type=int, default=256)
parser.add_argument('--K', type=int, default=5)
parser.add_argument('--KK', type=int, default=5)
parser.add_argument('--epsilon', type=float, default=1e-5)
parser.add_argument('--layer',
                    type=str,
                    default="wn",
                    choices=["linear", "wn"])
parser.add_argument('--classifier',
                    type=str,
                    default="bn",
                    choices=["ori", "bn"])
parser.add_argument('--output', type=str, default='weight/target/')
parser.add_argument('--output_src', type=str, default='weight/source/')
parser.add_argument('--tag', type=str, default='noself')
parser.add_argument('--da',
                    type=str,
                    default='uda')
parser.add_argument('--issave', type=bool, default=True)
args = parser.parse_args([])

if args.dset == 'office-home':
    names = ['Art', 'Clipart', 'Product', 'RealWorld']
    args.class_num = 65
if args.dset == 'visda-2017':
    names = ['train', 'validation']
    args.class_num = 12

os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
SEED = args.seed
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

for i in range(len(names)):
    if i == args.s:
        continue
    args.t = i

    folder = './data/'
    args.s_dset_path = folder + args.dset + '/' + names[
        args.s] + '_list.txt'
    args.t_dset_path = folder + args.dset + '/' + names[
        args.t] + '_list.txt'
    args.test_dset_path = folder + args.dset + '/' + names[
        args.t] + '_list.txt'

    args.output_dir_src = osp.join(args.output_src, args.da, args.dset,
                                    names[args.s][0].upper())
    args.output_dir = osp.join(
        args.output, args.da, args.dset,
        names[args.s][0].upper() + names[args.t][0].upper())
    args.name = names[args.s][0].upper() + names[args.t][0].upper()

In [7]:
dset_loaders = data_load(args)
## set base network
netF = network.ResBase(res_name=args.net).cuda()

netB = network.feat_bootleneck(type=args.classifier,
                                feature_dim=netF.in_features,
                                bottleneck_dim=args.bottleneck).cuda()
netC = network.feat_classifier(type=args.layer,
                                class_num=args.class_num,
                                bottleneck_dim=args.bottleneck).cuda()

modelpath = args.output_dir + '/target_F_tar_5.pt'
netF.load_state_dict(torch.load(modelpath))
modelpath = args.output_dir + '/target_B_tar_5.pt'
netB.load_state_dict(torch.load(modelpath))
modelpath = args.output_dir + '/target_C_tar_5.pt'
netC.load_state_dict(torch.load(modelpath))

<All keys matched successfully>

In [8]:
netF.eval()
netB.eval()
netC.eval()
if args.dset == 'visda-2017':
    acc_s_te, acc_list = cal_acc(dset_loaders['test'], netF, netB,
                                    netC,flag= True)
print(acc_s_te)